In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt

# Define functions for loading data, training model, and making predictions
def load_data(ticker):
    try:
        data = yf.download(tickers=ticker, period='3mo', interval='1h')
        data['Direction'] = np.where(data['Close'] > data['Open'], 1, 0)
        return data
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return pd.DataFrame()

def preprocess_data(data):
    close_price = data['Close'].values
    close_price = close_price.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_price)
    return scaled_data, scaler

def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

def build_model(time_step):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def predict_next_24_hours(model, last_data, scaler):
    predictions = []
    directions = []
    input_data = last_data.copy()

    for _ in range(24):  # Predict for the next 24 hours
        pred = model.predict(input_data)
        pred_unscaled = scaler.inverse_transform(pred)
        predictions.append(pred_unscaled[0][0])

        # Determine direction: up (1) if the price is predicted to rise, down (0) otherwise
        last_price = scaler.inverse_transform(input_data[0])[-1][0]
        direction = "UP" if pred_unscaled[0][0] > last_price else "DOWN"
        directions.append(direction)

        # Update input data by appending the predicted price
        new_input = np.append(input_data[0][1:], pred, axis=0)
        input_data = new_input.reshape(1, -1, 1)

    return predictions, directions
